## Imports i definició de les variables d'arxius
---

In [17]:

import re #Regexs
import os #Acces a fitxers

import nltk #Tokenització
nltk.download('punkt') #Tokenització

import numpy as np #Matriu de confusió
import seaborn as sns #Matriu de confusió
import matplotlib.pyplot as plt #Matriu de confusió
import pandas as pd #Matriu de confusió

train_files_list = [f for f in os.listdir('corpora') if re.search(r'_trn\.txt$', f)]
test_files_list = [f for f in os.listdir('corpora') if re.search(r'_tst\.txt$', f)]

print(train_files_list)


['deu_trn.txt', 'eng_trn.txt', 'fra_trn.txt', 'ita_trn.txt', 'nld_trn.txt', 'spa_trn.txt']


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Preprocessing
---

· Eliminar digits del text 
· Convertir tot el text a minuscula
· Eliminar caracters especials
· Substitueix els espais en blanc continus per un de sol
· Concatena totes les frases amb un espai doble al mig


In [18]:

def preprocessing(text):
    text = text.lower() # Converteix a minúscules
    text = re.sub(r'[^\w\s]', '', text) # Elimina signes de puntuació
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text) # Elimina caracters especials
    text = re.sub(r'\d+', '', text) # Elimina números
    text = re.sub(r'[^\S\n]+', ' ', text) # Elimina espais en blanc sobrants excepte \n
    text = re.sub(r'\n', '  ', text) # Substitueix \n per dos espais en blanc
    text = re.sub(r' +', ' ',) # Elimina espais en blanc al principi i al final
    
    return text


# Creació dels trigrames de caracters, amb la seva freqüència
---

In [28]:

def character_trigrams_finder(text):
    finder = nltk.TrigramCollocationFinder.from_words(text)
    finder.apply_freq_filter(5) # Filtra els trigrames que apareixen menys de 5 vegades
    trigrams = finder.ngram_fd.items()
    trigrams = sorted(trigrams, key=lambda x: x[1], reverse=True)

    return trigrams

In [29]:
def word_trigrams_finder(text):
    finder = nltk.TrigramCollocationFinder.from_words(nltk.word_tokenize(text))
    finder.apply_freq_filter(5) # Filtra els trigrames amb freqüència menor a 5
    trigrams = finder.ngram_fd.items()
    trigrams = sorted(trigrams, key=lambda x: x[1], reverse=True)

    return trigrams

In [30]:
def trigram_assigner(train_files_list):
    
    enclosed_trigrams_ch = {}
    enclosed_trigrams_wds = {}

    for file in train_files_list:
        with open('corpora/' + file, 'r', encoding='utf-8') as f:
            language_id = file.split('_')[0] # ID de l'idioma
            print(language_id)
            text = f.read()
            text = preprocessing(text)
            
            enclosed_trigrams_ch[language_id] = character_trigrams_finder(text)
            enclosed_trigrams_wds[language_id] = word_trigrams_finder(text)
            
    return enclosed_trigrams_ch, enclosed_trigrams_wds


In [31]:
trigram_assigner(train_files_list)

deu
eng
fra
ita
nld
spa


({'deu': [(('e', 'n', ' '), 72299),
   (('e', 'r', ' '), 45659),
   ((' ', ' ', ' '), 30365),
   ((' ', 'd', 'e'), 30147),
   (('d', 'e', 'r'), 23029),
   (('i', 'e', ' '), 22724),
   (('i', 'c', 'h'), 21335),
   (('e', 'i', 'n'), 21119),
   (('s', 'c', 'h'), 20400),
   ((' ', 'd', 'i'), 19938),
   (('d', 'i', 'e'), 19806),
   (('c', 'h', ' '), 19532),
   (('i', 'n', ' '), 16110),
   (('n', 'd', ' '), 16089),
   (('n', ' ', 'd'), 15158),
   ((' ', 'e', 'i'), 15015),
   (('c', 'h', 'e'), 14996),
   ((' ', 'u', 'n'), 14595),
   (('d', 'e', 'n'), 14231),
   (('t', 'e', 'n'), 13743),
   ((' ', 'a', 'u'), 13621),
   ((' ', 'd', 'a'), 13574),
   (('u', 'n', 'd'), 13388),
   ((' ', 'b', 'e'), 12259),
   (('t', 'e', ' '), 12217),
   (('i', 'n', 'e'), 12091),
   (('g', 'e', 'n'), 12056),
   ((' ', 'i', 'n'), 12050),
   (('c', 'h', 't'), 12043),
   ((' ', 'g', 'e'), 11990),
   (('n', ' ', ' '), 11417),
   (('t', 'e', 'r'), 10872),
   (('u', 'n', 'g'), 10807),
   (('e', 's', ' '), 10782),
   (('n

## Separar el test per oracions
---

In [23]:

def test_files_separator(test_files_list):
    test_files = {}
    for file in test_files_list:
        with open('corpora/' + file, 'r', encoding='utf-8') as f:
            language_id = file.split('_')[0] # ID de l'idioma
            print(language_id)
            text = f.read()
            text = preprocessing(text)
            test_files[language_id] = nltk.sent_tokenize(text)
    return test_files

In [ ]:
test_files_separator(test_files_list)